# Regions with Convolutional Neural Networks model for image segmentation:

In [1]:
from bowl_config import bowl_config
from bowl_dataset import BowlDataset
import utils
import model as modellib
from model import log
from glob import glob


Configurations:
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        512
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.0001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               256
MEAN_PIXEL                     [ 123.7  116.8  103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           bowl
NUM_CLASSES                    2
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING  

Using TensorFlow backend.


In [2]:
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm
from inference_config import inference_config
from utils import rle_encode, rle_decode, rle_to_string
import functions as f

In [3]:
import os
import time

## Root directory of the project:

In [4]:
ROOT_DIR = os.getcwd()

## Directory to save logs and trained model:

In [5]:
MODEL_DIR = os.path.join(ROOT_DIR, "model/checkpoint")

In [6]:
gen_predictions = True
train_model = False

if gen_predictions:
    model = modellib.MaskRCNN(mode="inference", 
                              config=inference_config,
                              model_dir=MODEL_DIR)
elif train_model:
    model = modellib.MaskRCNN(mode="training", 
                              config=bowl_config,
                              model_dir=MODEL_DIR)

In [7]:
CHECKPOINT_PATH = os.path.join(MODEL_DIR, 'RCNN_checkpoint.h5')

In [8]:
if train_model !=False:
    # Training dataset
    dataset_train = BowlDataset()
    dataset_train.load_bowl('./fixed_data/train')
    dataset_train.prepare()

In [9]:
if train_model != False:
    # Validation dataset
    dataset_val = BowlDataset()
    dataset_val.load_bowl('./fixed_data/train')
    dataset_val.prepare()

In [10]:
TEST_DIR = './data/test/'

In [11]:
if gen_predictions:
    # Test dataset
    dataset_test = BowlDataset()
    dataset_test.load_bowl(TEST_DIR)
    dataset_test.prepare()

### Build RCNN model:

In [12]:
if train_model !=False:
    # Which weights to start with?
    init_with = "last"  # imagenet, coco, checkpoint or last

In [13]:
if train_model !=False:
    if init_with == "imagenet":
        model.load_weights(model.get_imagenet_weights(), by_name=True)
    elif init_with == "coco":
        # Load weights trained on MS COCO, but skip layers that
        # are different due to the different number of classes
        # See README for instructions to download the COCO weights
        model.load_weights(COCO_MODEL_PATH, by_name=True,
                           exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                    "mrcnn_bbox", "mrcnn_mask"])
    elif init_with == "checkpoint":
        model.load_weights(CHECKPOINT_PATH, by_name=False)
    elif init_with == "last":
        # Load the last model you trained and continue training
        model.load_weights(model.find_last()[1], by_name=True)
    
    model.train(dataset_train,dataset_val, 
                learning_rate=bowl_config.LEARNING_RATE / 10,
                epochs=2, 
                layers="all")
elif gen_predictions:
    # Recreate the model in inference mode
    model = modellib.MaskRCNN(mode="inference", 
                              config=inference_config,
                              model_dir=MODEL_DIR)
    model_path = model.find_last()[1]

    # Load trained weights (fill in path to trained weights here)
    assert model_path != "", "Provide path to trained weights"
    print("Loading weights from ", model_path)
    model.load_weights(model_path, by_name=True)

Loading weights from  /home/u12162/model/checkpoint/bowl20180411T0019/mask_rcnn_bowl_0002.h5


In [14]:
SUBMISSION_FILE = './stage1_sample_submission.csv'

def generate_RCNN_prediction(model = None, \
                             SUBMISSION_FILE = None, \
                             TEST_DIR = None, \
                             OUTPUT_FILE = None):
    
    output = []
    sample_submission = pd.read_csv(SUBMISSION_FILE)
    
    ImageId = []
    EncodedPixels = []
    
    for image_id in tqdm(sample_submission.ImageId):
        image_path = os.path.join(TEST_DIR, image_id, 'images', image_id + '.png')
    
        original_image = cv2.imread(image_path)
        results = model.detect([original_image], verbose=0)
        r = results[0]
    
        masks = r['masks']
        
        ImageId_batch, EncodedPixels_batch = f.numpy2encoding_no_overlap2(masks, image_id, r['scores'])
        ImageId += ImageId_batch
        EncodedPixels += EncodedPixels_batch
    
    f.write2csv(OUTPUT_FILE, ImageId, EncodedPixels)

In [15]:
def generate_timestamp():
    timestring = time.strftime("%Y_%m_%d-%H_%M_%S")
    print ("Time stamp generated: "+timestring)
    return timestring

In [16]:
timestr = generate_timestamp()

Time stamp generated: 2018_04_11-04_30_16


In [17]:
generate_RCNN_prediction(model = model,
                         SUBMISSION_FILE = SUBMISSION_FILE,
                         TEST_DIR = TEST_DIR,
                         OUTPUT_FILE = os.path.join('./sub-dsbowl2018-RCNN_'+ timestr + '.csv'))

100%|██████████| 65/65 [05:22<00:00,  4.96s/it]
